In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
# In this cell, Train and Test sets are loaded
train=pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv')
test = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')

In [ ]:
train.head()

In [ ]:
test.head()

### From the train set, we can observe that the feature 'date' is trivial as it infers the same as the feature 'date_block_num', hence i will drop this 'date' feature.
### I will also drop the feature 'item_price' as it is not given in the test data, so what's the purpose of training our model considering this feature if we are not going to make predictions using this feature.

In [ ]:
train['item_cnt_day']=abs(train['item_cnt_day'])

In [ ]:
train.drop(columns=['date','item_price'],axis=1,inplace=True)
train.drop_duplicates(inplace=True,keep='first',ignore_index=True)

In [ ]:
train

### Here the Target feature is 'item_cnt_day' which is a continuous quantity.
### But the independent features ['shop_id','item_id'] are discrete/labeled...
### Hence we go for Pivoting to get the no. of items sold per month for the respective ['shop_id','item_id']

In [ ]:
train1=pd.pivot_table(train,index=['shop_id','item_id'],columns='date_block_num',values='item_cnt_day',aggfunc=np.sum).reset_index()
train1
# Here the columns 0,1...33 have the items_per_month values

### In below step, we then merge train1 and test to convert the test[['shop_id','item_id']] into the number of the respected items/sold/month for the past 34 months.

In [ ]:
test1=test.merge(train1,how='left',on=['shop_id','item_id']).drop(columns=['shop_id','item_id']).fillna(value=0)
test1

In [ ]:
x_train=test1.iloc[:,1:-1]
y_train=test1.iloc[:,-1]
x_test=test1.iloc[:,2:]

In [ ]:
x_train

In [ ]:
y_train

In [ ]:
x_test

In [ ]:
lr=LinearRegression()
lr.fit(x_train,y_train)
print('Linear Regression model score is:',lr.score(x_train,y_train))
print('MSE: ',mean_squared_error(y_train,lr.predict(x_train)))
lr.predict(x_test)

In [ ]:
dr=DecisionTreeRegressor()
dr.fit(x_train,y_train)
print('DecisionTreeRegressor model score is:',dr.score(x_train,y_train))
print('MSE: ',mean_squared_error(y_train,dr.predict(x_train)))
dr.predict(x_test)

In [ ]:
gb=GradientBoostingRegressor()
gb.fit(x_train,y_train)
print('GradientBoostRegressor model score is:',gb.score(x_train,y_train))
print('MSE: ',mean_squared_error(y_train,gb.predict(x_train)))
y_pred=gb.predict(x_test)

In [ ]:
rr=RandomForestRegressor()
rr.fit(x_train,y_train)
print('RandomForestRegressor model score is:',rr.score(x_train,y_train))
print('MSE: ',mean_squared_error(y_train,rr.predict(x_train)))
rr.predict(x_test)

In [ ]:
output = pd.DataFrame({'Id':test1.iloc[:,0],'item_cnt_month': [abs(i) for i in y_pred.tolist()]})
output.to_csv('mysubmission.csv', index=False)

Here i am submitting the GradientBoostingRegressor predictions as it is giving the best score among the above all.